In [15]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import re
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import os
import time
import unidecode

from numpy import array
from numpy import argmax
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
import math
import operator
import warnings
warnings.filterwarnings('ignore')
import random

import requests
from requests.structures import CaseInsensitiveDict
import json

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from keras.layers import Activation, Dense
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dropout

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

# NFL MERGING

In [16]:
off_PBP_partic = pd.read_csv("PBP Data/off_participatio_pbp.csv", index_col = False)

<h3> Map Passing <h3>

In [19]:
mappingTable = pd.read_csv("mappingTable.csv", index_col = False)
passing_pbp = pd.read_csv("PBP Data/passing_pbp2021.csv", index_col = False)
#passing_pbp = pd.read_csv("PBP Data/passing_pbp.csv", index_col = False)
roster = pd.read_csv("roster.csv", index_col = False)
#roster = roster[roster['season'] == 2020].reset_index(drop = False)
roster['playerName'] = roster['firstName'] + " " + roster['lastName']

roster = pd.read_csv("roster.csv", index_col = False)
#roster = roster[roster['season'] == 2020].reset_index(drop = False)
roster['playerName'] = roster['firstName'] + " " + roster['lastName']

#roster.to_csv("Positions.csv", index = False)

# actionType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'ActionType'] #1
passing_pbp_map = pd.merge(passing_pbp, statMapping, left_on = ['actionType'],  #2
                          right_on=['Code'], how = 'left')
passing_pbp_map = passing_pbp_map.drop(['Stat','Code'], axis = 1)
passing_pbp_map = passing_pbp_map.rename(columns={'descr' : 'ActionType_descr'}) #3


#throwType mapping

throwTypeMapping = mappingTable[mappingTable['Stat'] == 'ThrowType']
passing_pbp_map = pd.merge(passing_pbp_map, throwTypeMapping, left_on = ['throwType'], 
                          right_on=['Code'], how = 'left')
passing_pbp_map = passing_pbp_map.drop(['Stat','Code'], axis = 1)
passing_pbp_map = passing_pbp_map.rename(columns={'descr' : 'ThrowType_descr'})


#throwDirection mapping
throwDirectionMapping = mappingTable[mappingTable['Stat'] == 'ThrowDirection']
passing_pbp_map = pd.merge(passing_pbp_map, throwDirectionMapping, left_on = ['throwDirection'], 
                          right_on=['Code'], how = 'left')
passing_pbp_map = passing_pbp_map.drop(['Stat','Code'], axis = 1)
passing_pbp_map = passing_pbp_map.rename(columns={'descr' : 'ThrowDirection_descr'})


#throwAccuracy mapping
throwAccuracyMapping = mappingTable[mappingTable['Stat'] == 'ThrowAccuracy'] #1 #2
passing_pbp_map = pd.merge(passing_pbp_map, throwAccuracyMapping, left_on = ['throwAccuracy'],  #3 #4
                          right_on=['Code'], how = 'left')
passing_pbp_map = passing_pbp_map.drop(['Stat','Code'], axis = 1)
passing_pbp_map = passing_pbp_map.rename(columns={'descr' : 'ThrowAccuracy_descr'}) #5


#scoreType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'ScoringType'] #1
passing_pbp_map = pd.merge(passing_pbp_map, statMapping, left_on = ['scoreType'],  #2
                          right_on=['Code'], how = 'left')
passing_pbp_map = passing_pbp_map.drop(['Stat','Code'], axis = 1)
passing_pbp_map = passing_pbp_map.rename(columns={'descr' : 'ScoreType_descr'}) #3


#dropType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'DropType'] #1
passing_pbp_map = pd.merge(passing_pbp_map, statMapping, left_on = ['dropType'],  #2
                          right_on=['Code'], how = 'left')
passing_pbp_map = passing_pbp_map.drop(['Stat','Code'], axis = 1)
passing_pbp_map = passing_pbp_map.rename(columns={'descr' : 'DropType_descr'}) #3


#atPassType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'QBAtPass'] #1
passing_pbp_map = pd.merge(passing_pbp_map, statMapping, left_on = ['qbAtPassType'],  #2
                          right_on=['Code'], how = 'left')
passing_pbp_map = passing_pbp_map.drop(['Stat','Code'], axis = 1)
passing_pbp_map = passing_pbp_map.rename(columns={'descr' : 'AtPass_descr'}) #3

In [20]:
dropBack = []
for i in range(len(passing_pbp_map)):
    if (passing_pbp_map['dropType'][i] == 1) |(passing_pbp_map['dropType'][i] == 2) | (passing_pbp_map['dropType'][i] == 3) | (passing_pbp_map['dropType'][i] == 4):
        dropBack.append(1)
    else:
        dropBack.append(0)
passing_pbp_map['dropBack'] = dropBack
        

In [21]:
# Player name mapping
passing_pbp_map = pd.merge(passing_pbp_map, roster[['playerId', 'positionCategory', 'playerName']], left_on = ['playerId'],  #2
                          right_on=['playerId'], how = 'left')



<h3> Map Receiving <h3>

In [23]:
mappingTable = pd.read_csv("mappingTable.csv", index_col = False)
receiving_pbp = pd.read_csv("PBP Data/receiving_pbp2021.csv", index_col = False)
#receiving_pbp = pd.read_csv("PBP Data/receiving_pbp.csv", index_col = False)

In [24]:
# actionType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'ActionType'] #1
receiving_pbp_map = pd.merge(receiving_pbp, statMapping, left_on = ['actionType'],  #2
                          right_on=['Code'], how = 'left')
receiving_pbp_map = receiving_pbp_map.drop(['Stat','Code'], axis = 1)
receiving_pbp_map = receiving_pbp_map.rename(columns={'descr' : 'ActionType_descr'}) #3

# targetType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'TargetTypes'] #1
receiving_pbp_map = pd.merge(receiving_pbp_map, statMapping, left_on = ['targetType'],  #2
                          right_on=['Code'], how = 'left')
receiving_pbp_map = receiving_pbp_map.drop(['Stat','Code'], axis = 1)
receiving_pbp_map = receiving_pbp_map.rename(columns={'descr' : 'TargetType_descr'}) #3

# scoreType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'ScoringType'] #1
receiving_pbp_map = pd.merge(receiving_pbp_map, statMapping, left_on = ['scoreType'],  #2
                          right_on=['Code'], how = 'left')
receiving_pbp_map = receiving_pbp_map.drop(['Stat','Code'], axis = 1)
receiving_pbp_map = receiving_pbp_map.rename(columns={'descr' : 'ScoreType_descr'}) #3

# TargetLinedUp mapping
statMapping = mappingTable[mappingTable['Stat'] == 'TargetLinedUp'] #1
receiving_pbp_map = pd.merge(receiving_pbp_map, statMapping, left_on = ['targetLinedUp'],  #2
                          right_on=['Code'], how = 'left')
receiving_pbp_map = receiving_pbp_map.drop(['Stat','Code'], axis = 1)
receiving_pbp_map = receiving_pbp_map.rename(columns={'descr' : 'TargetLinedUp_descr'}) #3

# Target direction missing

In [25]:
# Add player name

receiving_pbp_map = pd.merge(receiving_pbp_map, roster[['playerId', 'positionCategory', 'playerName']], left_on = ['playerId'],  #2
                          right_on=['playerId'], how = 'left')



In [26]:
# Route type mapping from QB to the intended receiver.
passing_pbp_map = pd.merge(passing_pbp_map, receiving_pbp_map[['season','week','gameId','eventId','playerId', 'targetType','TargetType_descr']], 
                           left_on = ['season','week','gameId','eventId','intendedReceiverId'],  #2
                          right_on=['season','week','gameId','eventId','playerId'], how = 'left')

passing_pbp_map = passing_pbp_map.drop(columns = ['playerId_y'])
passing_pbp_map = passing_pbp_map.rename(columns={'playerId_x' : 'playerId'})


<h3> Map Rushing <h3>

In [27]:
# Mapping starts rushing
mappingTable = pd.read_csv("mappingTable.csv", index_col = False)
rushing_pbp = pd.read_csv("PBP Data/rushing_pbp2021.csv", index_col = False)
#rushing_pbp = pd.read_csv("PBP Data/rushing_pbp.csv", index_col = False)

In [28]:
# actionType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'ActionType'] #1
rushing_pbp_map = pd.merge(rushing_pbp, statMapping, left_on = ['actionType'],  #2
                          right_on=['Code'], how = 'left')
rushing_pbp_map = rushing_pbp_map.drop(['Stat','Code'], axis = 1)
rushing_pbp_map = rushing_pbp_map.rename(columns={'descr' : 'ActionType_descr'}) #3

# runType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'RunType'] #1
rushing_pbp_map = pd.merge(rushing_pbp_map, statMapping, left_on = ['runType'],  #2
                          right_on=['Code'], how = 'left')
rushing_pbp_map = rushing_pbp_map.drop(['Stat','Code'], axis = 1)
rushing_pbp_map = rushing_pbp_map.rename(columns={'descr' : 'RunType_descr'}) #3

# runDirection mapping
statMapping = mappingTable[mappingTable['Stat'] == 'RunDirection'] #1
rushing_pbp_map = pd.merge(rushing_pbp_map, statMapping, left_on = ['runDirection'],  #2
                          right_on=['Code'], how = 'left')
rushing_pbp_map = rushing_pbp_map.drop(['Stat','Code'], axis = 1)
rushing_pbp_map = rushing_pbp_map.rename(columns={'descr' : 'RunDirection_descr'}) #3

# scoreType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'ScoringType'] #1
rushing_pbp_map = pd.merge(rushing_pbp_map, statMapping, left_on = ['scoreType'],  #2
                          right_on=['Code'], how = 'left')
rushing_pbp_map = rushing_pbp_map.drop(['Stat','Code'], axis = 1)
rushing_pbp_map = rushing_pbp_map.rename(columns={'descr' : 'ScoreType_descr'}) #3

# actualRunDirection mapping
statMapping = mappingTable[mappingTable['Stat'] == 'RunDirection'] #1
rushing_pbp_map = pd.merge(rushing_pbp_map, statMapping, left_on = ['actualRunDirection'],  #2
                          right_on=['Code'], how = 'left')
rushing_pbp_map = rushing_pbp_map.drop(['Stat','Code'], axis = 1)
rushing_pbp_map = rushing_pbp_map.rename(columns={'descr' : 'ActualRunDirection_descr'}) #3

# qbDropType mapping 
statMapping = mappingTable[mappingTable['Stat'] == 'DropType'] #1
rushing_pbp_map = pd.merge(rushing_pbp_map, statMapping, left_on = ['qbDropType'],  #2
                          right_on=['Code'], how = 'left')
rushing_pbp_map = rushing_pbp_map.drop(['Stat','Code'], axis = 1)
rushing_pbp_map = rushing_pbp_map.rename(columns={'descr' : 'qbDropType_descr'}) #3

# blockingSchemeType mapping 
statMapping = mappingTable[mappingTable['Stat'] == 'blockingSchemeType'] #1
rushing_pbp_map = pd.merge(rushing_pbp_map, statMapping, left_on = ['blockingSchemeType'],  #2
                          right_on=['Code'], how = 'left')
rushing_pbp_map = rushing_pbp_map.drop(['Stat','Code'], axis = 1)
rushing_pbp_map = rushing_pbp_map.rename(columns={'descr' : 'blockingSchemeType_descr'}) #3

In [29]:
# Add player name

rushing_pbp_map = pd.merge(rushing_pbp_map, roster[['playerId', 'positionCategory', 'playerName']], left_on = ['playerId'],  #2
                          right_on=['playerId'], how = 'left')



<h3> Map presnap defense <h3>

In [30]:
mappingTable = pd.read_csv("mappingTable.csv", index_col = False)
presnapDefense_pbp = pd.read_csv("PBP Data/preSnapDefense_pbp2021.csv", index_col = False)

#presnapDefense_pbp = pd.read_csv("PBP Data/preSnapDefense_pbp2020.csv", index_col = False)

In [31]:
# CoverageType mapping
statMapping = mappingTable[mappingTable['Stat'] == 'CoverageType'] #1
presnapDefense_pbp_map = pd.merge(presnapDefense_pbp, statMapping, left_on = ['coverageSchemeType'],  #2
                          right_on=['Code'], how = 'left')
presnapDefense_pbp_map = presnapDefense_pbp_map.drop(['Stat','Code'], axis = 1)
presnapDefense_pbp_map = presnapDefense_pbp_map.rename(columns={'descr' : 'CoverageType_descr'}) #3

In [32]:
# Load mapping for type 1 and type 3

cover1 = [16, 17, 18, 19]
cover2 = [20]
cover3 = [21, 22, 23, 24, 25, 26, 27, 28]

CoverageType_descr_tmp = []

for i in range(len(presnapDefense_pbp_map)):
    
    if presnapDefense_pbp_map['coverageSchemeType'][i] in cover1:
        CoverageType_descr_tmp.append("Cover 1")
        
    elif presnapDefense_pbp_map['coverageSchemeType'][i] in cover2:
        CoverageType_descr_tmp.append("Cover 2")
        
    elif presnapDefense_pbp_map['coverageSchemeType'][i] in cover3:
        CoverageType_descr_tmp.append("Cover 3")
    elif presnapDefense_pbp_map['coverageSchemeType'][i] in cover3:
        CoverageType_descr_tmp.append("Cover 6")
    else:
        CoverageType_descr_tmp.append(presnapDefense_pbp_map['CoverageType_descr'][i])
    

presnapDefense_pbp_map['CoverageType_descr'] = CoverageType_descr_tmp

In [33]:
#Coverage type pooling

CoverageType_pooled = []

for i in range(len(presnapDefense_pbp_map)):
    if presnapDefense_pbp_map['CoverageType_descr'][i] in ['Cover 0', 'Cover 1', 'Cover 2', 'Man Cover 2']:
        CoverageType_pooled.append("Man")
    elif presnapDefense_pbp_map['CoverageType_descr'][i] in ['Cover 3','Cover 4', 'Cover 6']:
        CoverageType_pooled.append("Zone")
    else:
        CoverageType_pooled.append("Other")
    
presnapDefense_pbp_map['CoverageType_pooled'] = CoverageType_pooled
    
#'Cover 0', 'Cover 1', 'Cover 2'


#cover 0, cover 1, man cover 2
#is Man

#the others that are numbered
#are zone



<h5> Team ID Table (from def participation) <h5>

In [35]:
teamNameMapping = pd.read_csv("teamNameMapping.csv", index_col = False)
teamNameMapping = teamNameMapping[['teamId', 'nickName']]

<h5> Load Pre Snap Mapping <h5>

In [36]:
preSnapDefense = pd.read_csv("PBP Data/preSnapDefense_pbp2021.csv", index_col = False)
#preSnapDefense = pd.read_csv("PBP Data/preSnapDefense_pbp2020.csv", index_col = False)
preSnapDefenseTeam = preSnapDefense[['season', 'week', 'gameId','teamId']].drop_duplicates()

preSnapDefenseTeam = preSnapDefenseTeam.rename(columns={'teamId' : 'oppTeamId'})
#teamNameMapping = teamNameMapping[['teamId', 'gsisid']]

In [37]:
# map team ID to team name
preSnapDefenseTeam = pd.merge(preSnapDefenseTeam, teamNameMapping, left_on = ['oppTeamId'], 
                          right_on=['teamId'], how = 'left')

<h5> DepthChart <h5>

In [55]:
depthChartOffense = pd.read_csv("DepthChart/DepthChartOffense.csv", index_col = False)
depthChartDefense = pd.read_csv("DepthChart/DepthChartDefense.csv", index_col = False)

# Passing

In [56]:
# Merge passing with coverage types
passing_pbp_coverage = pd.merge(passing_pbp_map, 
                                presnapDefense_pbp_map[['season', 'week', 'gameId', 'eventId', 'CoverageType_descr', 'CoverageType_pooled']], 
                                left_on = ['season', 'week', 'gameId', 'eventId'], 
                          right_on=['season', 'week', 'gameId', 'eventId'], how = 'left')


In [57]:
# Add FP_yards
passing_pbp_coverage['FP_yards'] = passing_pbp_coverage['yards'] * 0.04

# Add FP_TD
FP_TD= []
for i in range(len(passing_pbp_coverage)):
    if passing_pbp_coverage['scoreType'][i] == 6:
        FP_TD.append(4)
    else:
        FP_TD.append(0)
passing_pbp_coverage['FP_TD'] = FP_TD

# Add FP_intercept
FP_intercept= []
for i in range(len(passing_pbp_coverage)):
    if passing_pbp_coverage['intercepted'][i] == False:
        FP_intercept.append(0)
    else: 
        FP_intercept.append(-1)
passing_pbp_coverage['FP_intercept'] = FP_intercept

# Add FP_total
passing_pbp_coverage['FP_total'] = passing_pbp_coverage['FP_yards'] +  passing_pbp_coverage['FP_TD'] + passing_pbp_coverage['FP_intercept']



In [58]:
# Derive dropbacks

passing_pbp_coverage_DB = passing_pbp_coverage[(passing_pbp_coverage['actionType'] == 1) & (passing_pbp_coverage['dropBack'] == 1)].reset_index(drop=True)
#passing_pbp_coverage_DB = passing_pbp_coverage[(passing_pbp_coverage['dropBack'] == 1)].reset_index(drop=True)



In [63]:
# Passing by Drop Type - Map PBP to depth chart - Double/triple loops and LONG RUN TIME
depthChart = []
team = []
for i in range(len(passing_pbp_coverage_DB)):
    inDepthChart = 0
    team_tmp = 'team'
    for j in depthChartOffense.columns:
        for k in range(len(depthChartOffense)):
            if passing_pbp_coverage_DB['playerName'][i] == depthChartOffense[j][k]:
                inDepthChart += 1
                team_tmp = j
    if inDepthChart != 0:
        depthChart.append(1)
        team.append(team_tmp)
    else:
        depthChart.append(0)
        team.append("")
passing_pbp_coverage_DB['depthChart'] = depthChart
passing_pbp_coverage_DB['team'] = team
passing_pbp_coverage_DB = passing_pbp_coverage_DB[passing_pbp_coverage_DB['depthChart'] == 1].reset_index(drop = True)


KeyboardInterrupt: 

In [178]:
#passing_pbp_coverage_DB.to_csv("NN_Data/passing_pbp_coverage_082922.csv", index = False)

In [64]:
# Add # of play
passing_pbp_coverage_DB['plays'] = 1

#Group by drop type 

count = 0
for i in np.unique(passing_pbp_coverage_DB['playerName']):
    playerIDPassDT = passing_pbp_coverage_DB[passing_pbp_coverage_DB['playerName'] == i]
    if count == 0:
        dfFPDB = playerIDPassDT.groupby(by=["team","playerName","DropType_descr"]).sum()
    else:
        dfFPDB = dfFPDB.append(playerIDPassDT.groupby(by=["team","playerName","DropType_descr"]).sum())
    count += 1

dfFPDB.reset_index(inplace=True)

dfFPDB['FP/DB'] = dfFPDB['FP_total']/dfFPDB['dropBack']

dfFPDB = dfFPDB[['team','playerName','DropType_descr','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]

    
#passing_pbp_coverage.groupby(by=["CoverageType_descr"]).sum()


KeyError: 'team'

In [199]:
# Calculte frequency

totalPlayCountTable = dfFPDB.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB = pd.merge(dfFPDB, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
dfFPDB['freq'] = dfFPDB['plays'] / dfFPDB['totalPlays']

<h5> PASSING FP by drop type - defensive team <h5>

In [71]:
# Merge with opponent teams

passing_pbp_coverage_OPP = pd.merge(passing_pbp_coverage_DB, preSnapDefenseTeam[['season', 'week', 'gameId','oppTeamId', 'nickName']], 
         left_on = ['season', 'week', 'gameId'], 
                          right_on=['season', 'week', 'gameId'], how = 'left')
passing_pbp_coverage_OPP = passing_pbp_coverage_OPP[passing_pbp_coverage_OPP['teamId'] != passing_pbp_coverage_OPP['oppTeamId']].reset_index(drop = True)

passing_pbp_coverage_OPP['plays'] = 1
# passing_pbp_coverage_OPP.to_csv("NN_Data/Defense/passing_pbp_coverage_OPP_081022.csv", index = False)


In [73]:
#Group by defensive team and drop type 

count = 0
for i in np.unique(passing_pbp_coverage_OPP['nickName']):
    playerIDPassDT = passing_pbp_coverage_OPP[passing_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        dfFPDB_def = playerIDPassDT.groupby(by=["nickName","DropType_descr"]).sum()

    else:
        dfFPDB_def = dfFPDB_def.append(playerIDPassDT.groupby(by=["nickName","DropType_descr"]).sum())
    count += 1

dfFPDB_def.reset_index(inplace=True)

dfFPDB_def['FP/DB'] = dfFPDB_def['FP_total']/dfFPDB_def['dropBack']
    
dfFPDB_def = dfFPDB_def[['nickName','DropType_descr','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]

    


In [74]:
# Calculte frequency

totalPlayCountTable = dfFPDB_def.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB_def = pd.merge(dfFPDB_def, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
dfFPDB_def['freq'] = dfFPDB_def['plays'] / dfFPDB_def['totalPlays']

<h5> Passing - Group by defense coverage <h5>

In [1072]:
count = 0
for i in np.unique(passing_pbp_coverage_DB['playerName']):
    playerIDPassDT = passing_pbp_coverage_DB[passing_pbp_coverage_DB['playerName'] == i]
    if count == 0:
        dfFPDB_CoverageType = playerIDPassDT.groupby(by=["team","playerName","CoverageType_descr"]).sum()
        
    else:
        dfFPDB_CoverageType = dfFPDB_CoverageType.append(playerIDPassDT.groupby(by=["team","playerName","CoverageType_descr"]).sum())
    count += 1
    
dfFPDB_CoverageType.reset_index(inplace=True)

dfFPDB_CoverageType['FP/DB'] = dfFPDB_CoverageType['FP_total']/dfFPDB_CoverageType['dropBack']

dfFPDB_CoverageType = dfFPDB_CoverageType[['team', 'playerName','CoverageType_descr','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]



In [1073]:
# Calculte frequency

totalPlayCountTable = dfFPDB_CoverageType.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB_CoverageType = pd.merge(dfFPDB_CoverageType, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
dfFPDB_CoverageType['freq'] = dfFPDB_CoverageType['plays'] / dfFPDB_CoverageType['totalPlays']

<h5> PASSING group by coverage - defensive team <h5>

In [1075]:
# FP by defense coverage

count = 0
for i in np.unique(passing_pbp_coverage_OPP['nickName']):
    playerIDPassDT = passing_pbp_coverage_OPP[passing_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        dfFPDB_CoverageType_def = playerIDPassDT.groupby(by=["nickName","CoverageType_descr"]).sum()
        
    else:
        dfFPDB_CoverageType_def = dfFPDB_CoverageType_def.append(playerIDPassDT.groupby(by=["nickName","CoverageType_descr"]).sum())
    count += 1
    
dfFPDB_CoverageType_def.reset_index(inplace=True)

dfFPDB_CoverageType_def['FP/DB'] = dfFPDB_CoverageType_def['FP_total']/dfFPDB_CoverageType_def['dropBack']

dfFPDB_CoverageType_def = dfFPDB_CoverageType_def[['nickName','CoverageType_descr','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]



In [1076]:

# Calculte frequency

totalPlayCountTable = dfFPDB_CoverageType_def.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB_CoverageType_def = pd.merge(dfFPDB_CoverageType_def, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
dfFPDB_CoverageType_def['freq'] = dfFPDB_CoverageType_def['plays'] / dfFPDB_CoverageType_def['totalPlays']

<h5> PASSING group by defense coverage - POOLED <h5>

In [1077]:
count = 0
for i in np.unique(passing_pbp_coverage_DB['playerName']):
    playerIDPassDT = passing_pbp_coverage_DB[passing_pbp_coverage_DB['playerName'] == i]
    if count == 0:
        dfFPDB_CoverageType_pooled = playerIDPassDT.groupby(by=["team","playerName","CoverageType_pooled"]).sum()
        
    else:
        dfFPDB_CoverageType_pooled = dfFPDB_CoverageType_pooled.append(playerIDPassDT.groupby(by=["team","playerName","CoverageType_pooled"]).sum())
    count += 1
    
dfFPDB_CoverageType_pooled.reset_index(inplace=True)

dfFPDB_CoverageType_pooled['FP/DB'] = dfFPDB_CoverageType_pooled['FP_total']/dfFPDB_CoverageType_pooled['dropBack']

dfFPDB_CoverageType_pooled = dfFPDB_CoverageType_pooled[['team', 'playerName','CoverageType_pooled','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]



In [1078]:
# Calculte frequency

totalPlayCountTable = dfFPDB_CoverageType_pooled.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB_CoverageType_pooled = pd.merge(dfFPDB_CoverageType_pooled, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
dfFPDB_CoverageType_pooled['freq'] = dfFPDB_CoverageType_pooled['plays'] / dfFPDB_CoverageType_pooled['totalPlays']

<h5> PASSING group by defense coverage - POOLED - defensive team <h5>

In [1079]:
# Group by defense coverage

count = 0
for i in np.unique(passing_pbp_coverage_OPP['nickName']):
    playerIDPassDT = passing_pbp_coverage_OPP[passing_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        dfFPDB_CoverageType_def_pooled = playerIDPassDT.groupby(by=["nickName","CoverageType_pooled"]).sum()
        
    else:
        dfFPDB_CoverageType_def_pooled = dfFPDB_CoverageType_def_pooled.append(playerIDPassDT.groupby(by=["nickName","CoverageType_pooled"]).sum())
    count += 1
    
dfFPDB_CoverageType_def_pooled.reset_index(inplace=True)

dfFPDB_CoverageType_def_pooled['FP/DB'] = dfFPDB_CoverageType_def_pooled['FP_total']/dfFPDB_CoverageType_def_pooled['dropBack']

dfFPDB_CoverageType_def_pooled = dfFPDB_CoverageType_def_pooled[['nickName','CoverageType_pooled','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]



In [1080]:

# Calculte frequency

totalPlayCountTable = dfFPDB_CoverageType_def_pooled.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB_CoverageType_def_pooled = pd.merge(dfFPDB_CoverageType_def_pooled, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
dfFPDB_CoverageType_def_pooled['freq'] = dfFPDB_CoverageType_def_pooled['plays'] / dfFPDB_CoverageType_def_pooled['totalPlays']


<h5> PASSING FP by target type <h5>

In [1083]:
# Mapping to numbering before grouping
passing_RouteTree = passing_pbp_coverage_DB.copy()
routeTree = pd.read_csv("RouteTree.csv", index_col = False)

targetTypeMap = []
for i in range(len(passing_RouteTree)):
    try:
        ind = passing_RouteTree['TargetType_descr'][i].index("-")
        targetTypeMap.append(passing_RouteTree['TargetType_descr'][i][:ind - 1])
    except:
        targetTypeMap.append(passing_RouteTree['TargetType_descr'][i])
passing_RouteTree['targetTypeMap'] = targetTypeMap

passing_RouteTree = pd.merge(passing_RouteTree, routeTree[['All', 'SIS']], left_on = ['targetTypeMap'],
                          right_on=['All'], how = 'left')

passing_RouteTree = passing_RouteTree.rename(columns={"SIS" : "route"})
passing_RouteTree = passing_RouteTree.drop(columns = ['All'])

In [1084]:
# Group by target type

count = 0
for i in np.unique(passing_RouteTree['playerName']):
    playerIDPassDT = passing_RouteTree[passing_RouteTree['playerName'] == i]
    if count == 0:
        dfFPDB_RouteType = playerIDPassDT.groupby(by=["team","playerName","route"]).sum()

    else:
        dfFPDB_RouteType = dfFPDB_RouteType.append(playerIDPassDT.groupby(by=["team","playerName","route"]).sum())
    count += 1

dfFPDB_RouteType.reset_index(inplace=True)

dfFPDB_RouteType['FP/DB'] = dfFPDB_RouteType['FP_total']/dfFPDB_RouteType['dropBack']


dfFPDB_RouteType = dfFPDB_RouteType[["team",'playerName', 'route','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]



In [1085]:

# Calculte frequency

totalPlayCountTable = dfFPDB_RouteType.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB_RouteType = pd.merge(dfFPDB_RouteType, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
dfFPDB_RouteType['freq'] = dfFPDB_RouteType['plays'] / dfFPDB_RouteType['totalPlays']

<h5> Group by coverage - defensive team <h5>

In [1086]:
passing_pbp_coverage_OPP

# Mapping to numbering first
passing_RouteTree = passing_pbp_coverage_OPP.copy()
routeTree = pd.read_csv("RouteTree.csv", index_col = False)

targetTypeMap = []
for i in range(len(passing_RouteTree)):
    try:
        ind = passing_RouteTree['TargetType_descr'][i].index("-")
        targetTypeMap.append(passing_RouteTree['TargetType_descr'][i][:ind - 1])
    except:
        targetTypeMap.append(passing_RouteTree['TargetType_descr'][i])
passing_RouteTree['targetTypeMap'] = targetTypeMap

passing_RouteTree = pd.merge(passing_RouteTree, routeTree[['All', 'SIS']], left_on = ['targetTypeMap'],
                          right_on=['All'], how = 'left')

passing_RouteTree = passing_RouteTree.rename(columns={"SIS" : "route"})
passing_RouteTree = passing_RouteTree.drop(columns = ['All'])

In [1087]:
count = 0
for i in np.unique(passing_RouteTree['nickName']):
    playerIDPassDT = passing_RouteTree[passing_RouteTree['nickName'] == i]
    if count == 0:
        dfFPDB_RouteType_def = playerIDPassDT.groupby(by=["nickName","route"]).sum()

    else:
        dfFPDB_RouteType_def = dfFPDB_RouteType_def.append(playerIDPassDT.groupby(by=["nickName","route"]).sum())
    count += 1

dfFPDB_RouteType_def.reset_index(inplace=True)

dfFPDB_RouteType_def['FP/DB'] = dfFPDB_RouteType_def['FP_total']/dfFPDB_RouteType_def['dropBack']


dfFPDB_RouteType_def = dfFPDB_RouteType_def[['nickName', 'route','dropBack' ,'FP_yards', 'FP_TD','FP_intercept','FP_total','FP/DB', 'plays']]



In [1088]:
# Calculte frequency

totalPlayCountTable = dfFPDB_RouteType_def.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
dfFPDB_RouteType_def = pd.merge(dfFPDB_RouteType_def, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
dfFPDB_RouteType_def['freq'] = dfFPDB_RouteType_def['plays'] / dfFPDB_RouteType_def['totalPlays']


In [1090]:
dfFPDB.to_csv('Split/Passing_DropType_2020_0817.csv')
dfFPDB_CoverageType.to_csv('Split/Passing_CoverageType_2020_0817.csv')
dfFPDB_CoverageType_pooled.to_csv('Split/Passing_CoverageType_pooled_2020_0817.csv')
dfFPDB_RouteType.to_csv('Split/Passing_RouteType_2020_0817.csv')

In [1091]:
dfFPDB_def.to_csv('Split/Passing_DropType_def2020__0817.csv')
dfFPDB_CoverageType_def.to_csv('Split/Passing_CoverageType_def2020__0817.csv')
dfFPDB_CoverageType_def_pooled.to_csv('Split/Passing_CoverageType_def_pooled2020__0817.csv')
dfFPDB_RouteType_def.to_csv('Split/Passing_RouteTyoe_def2020__0817.csv')

# Receiving

In [204]:
# Merge receiving with coverage types
receiving_pbp_coverage = pd.merge(receiving_pbp_map, 
                                presnapDefense_pbp_map[['season', 'week', 'gameId', 'eventId', 'coverageSchemeType', 'CoverageType_descr', 'CoverageType_pooled']], 
                                left_on = ['season', 'week', 'gameId', 'eventId'], 
                          right_on=['season', 'week', 'gameId', 'eventId'], how = 'left')



In [205]:
#  map to drop type from passing data

receiving_pbp_coverage = pd.merge(receiving_pbp_coverage, 
                                passing_pbp_map[['season', 'week', 'gameId', 'eventId', 'dropType', 'DropType_descr']], 
                                left_on = ['season', 'week', 'gameId', 'eventId'], 
                          right_on=['season', 'week', 'gameId', 'eventId'], how = 'left')


In [206]:
#Ensure the selected year is included
receiving_pbp_coverage= receiving_pbp_coverage[receiving_pbp_coverage['season'] == 2021].reset_index(drop = True)
#receiving_pbp_coverage= receiving_pbp_coverage[receiving_pbp_coverage['season'] == 2020].reset_index(drop = True)


In [207]:
# Clean up float

playerName = []
for i in range(len(receiving_pbp_coverage)):
    playerName.append(str(receiving_pbp_coverage['playerName'][i]))
receiving_pbp_coverage['playerName'] = playerName

In [209]:
# Add FP yards and touchdown
receiving_pbp_coverage['FP_yards'] = receiving_pbp_coverage['yards'] * 0.1

FP_TD= []
#passing_pbp_coverage['FP_TD'] = 0
for i in range(len(receiving_pbp_coverage)):
    if receiving_pbp_coverage['scoreType'][i] == 6:
        FP_TD.append(6)
    else:
        FP_TD.append(0)
receiving_pbp_coverage['FP_TD'] = FP_TD



In [210]:
# FP Catch/receiving
FP_receiving = []

for i in range(len(receiving_pbp_coverage)):
    if receiving_pbp_coverage['actionType'][i] == 2:
        FP_receiving.append(1)
    else: 
        FP_receiving.append(0)
receiving_pbp_coverage['FP_receiving'] = FP_receiving

In [211]:
receiving_pbp_coverage['FP_total'] = receiving_pbp_coverage['FP_yards'] +  receiving_pbp_coverage['FP_TD'] + receiving_pbp_coverage['FP_receiving']
receiving_pbp_coverage['plays'] = 1


In [65]:
# Receiving map to depth chart - LONG RUN TIME
depthChart = []
team = []
for i in range(len(receiving_pbp_coverage)):
    inDepthChart = 0
    team_tmp = 'team'
    for j in depthChartOffense.columns:
        for k in range(len(depChartOffense)):
            if receiving_pbp_coverage['playerName'][i] == depthChartOffense[j][k]:
                inDepthChart += 1
                team_tmp = j
    if inDepthChart != 0:
        depthChart.append(1)
        team.append(team_tmp)
    else:
        depthChart.append(0)
        team.append("")
receiving_pbp_coverage['depthChart'] = depthChart
receiving_pbp_coverage['team'] = team
receiving_pbp_coverage = receiving_pbp_coverage[receiving_pbp_coverage['depthChart'] == 1].reset_index(drop = True)


NameError: name 'receiving_pbp_coverage' is not defined

In [353]:
#receiving_pbp_coverage.to_csv("NN_Data/PBP_NN/receiving_pbp_081622.CSV", index = False)

In [214]:
# FP/route group by defense coverage

count = 0
for i in np.unique(receiving_pbp_coverage['playerName']):
    playerData = receiving_pbp_coverage[receiving_pbp_coverage['playerName'] == i]
    if count == 0:
        group = playerData.groupby(by=["team", "playerName","CoverageType_descr"]).sum()
        
    else:
        group = group.append(playerData.groupby(by=["team", "playerName","CoverageType_descr"]).sum())
    count += 1
    
group.reset_index(inplace=True)
receiving_CoverageType = group[["team", 'playerName','CoverageType_descr','FP_yards', 'FP_TD','FP_receiving','FP_total', 'plays']]

    

In [69]:
# Load all routes by coverage type (downloads from SIS dashboard)
directory = 'SIS Dashboard/CoverageType/'
#directory = 'SIS Dashboard/2020Archive/CoverageType/'
count = 0
for filename in os.listdir(directory):
    if 'csv' not in filename:
        continue
    f = os.path.join(directory, filename)
    fname = filename[:filename.index('.csv')]
    if count == 0:
        dfRead_CoverageType = pd.read_csv(f) 
        dfRead_CoverageType['CoverageType'] = fname
    else:
        dfRead_CoverageType_tmp = pd.read_csv(f)
        dfRead_CoverageType_tmp['CoverageType'] = fname
        dfRead_CoverageType = dfRead_CoverageType.append(dfRead_CoverageType_tmp, ignore_index=True)
    count += 1

In [217]:
# Merge receiving data with coverage type routes run.

receiving_CoverageType_routesRun = pd.merge(receiving_CoverageType, 
                                dfRead_CoverageType[['Player','CoverageType','Routes Run']], 
                                left_on = ['playerName','CoverageType_descr'], 
                          right_on=['Player','CoverageType', ], how = 'left')

receiving_CoverageType_routesRun = receiving_CoverageType_routesRun.dropna(subset = ['Routes Run'])

receiving_CoverageType_routesRun['FP/routes'] = receiving_CoverageType_routesRun['FP_total']/receiving_CoverageType_routesRun['Routes Run']

receiving_CoverageType_routesRun = receiving_CoverageType_routesRun[["team", 'playerName','CoverageType_descr','FP_yards','FP_TD', 'FP_receiving' , 'FP_total', 'Routes Run', 'FP/routes', 'plays']]



In [218]:
# Calculte frequency

totalPlayCountTable = receiving_CoverageType_routesRun.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_CoverageType_routesRun = pd.merge(receiving_CoverageType_routesRun, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
receiving_CoverageType_routesRun['freq'] = receiving_CoverageType_routesRun['plays'] / receiving_CoverageType_routesRun['totalPlays']


In [1111]:
receiving_CoverageType_routesRun.to_csv('Split/Receiving_CoverageType_2020_0817.csv', index = False)

<h5> Group by coverage - defensive team <h5>

In [221]:
# Merge PBP with opponent teams 

receiving_pbp_coverage_OPP = pd.merge(receiving_pbp_coverage, preSnapDefenseTeam[['season', 'week', 'gameId','oppTeamId', 'nickName']], 
         left_on = ['season', 'week', 'gameId'], 
                          right_on=['season', 'week', 'gameId'], how = 'left')
receiving_pbp_coverage_OPP = receiving_pbp_coverage_OPP[receiving_pbp_coverage_OPP['teamId'] != receiving_pbp_coverage_OPP['oppTeamId']].reset_index(drop = True)
receiving_pbp_coverage_OPP['plays'] = 1


In [78]:
#receiving_pbp_coverage_OPP.to_csv("NN_Data/Defense/receiving_pbp_OPP_081022.csv", index = False)

,season,week,gameId,eventId,teamId,playerId,actionType,targetType,targetDirection,targetDepth,...,DropType_descr,FP_yards,FP_TD,FP_receiving,FP_total,plays,deepChart,team,oppTeamId,nickName
0,2020,1,2793,45,16,6499,2,44,3,-1,...,0/1 Step,0.6,0,1,1.6,1,1,ARI,13,Texans
1,2020,1,2793,50,16,7176,2,34,4,-4,...,0/1 Step,0.6,0,1,1.6,1,1,KC,13,Texans
2,2020,1,2793,55,16,374,2,4,10,5,...,0/1 Step,1.1,0,1,2.1,1,1,KC,13,Texans
3,2020,1,2793,60,16,4143,13,20,7,36,...,5 Step,0.0,0,0,0.0,1,1,LV,13,Texans
4,2020,1,2793,70,16,4143,13,32,5,-1,...,5 Step,0.0,0,0,0.0,1,1,LV,13,Texans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13837,2020,21,3061,865,16,374,2,4,7,15,...,5 Step,1.6,0,1,2.6,1,1,KC,30,Buccaneers
13838,2020,21,3061,875,16,6499,2,44,3,-5,...,3 Step,0.1,0,1,1.1,1,1,ARI,30,Buccaneers
13839,2020,21,3061,880,16,4143,2,5,5,4,...,0/1 Step,1.1,0,1,2.1,1,1,LV,30,Buccaneers
13840,2020,21,3061,890,16,3938,2,4,5,5,...,0/1 Step,0.4,0,1,1.4,1,1,MIA,30,Buccaneers


In [223]:
# FP/route by defense coverage - defensive teams

count = 0
for i in np.unique(receiving_pbp_coverage_OPP['nickName']):
    playerData = receiving_pbp_coverage_OPP[receiving_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        group_def = playerData.groupby(by=["nickName","CoverageType_descr"]).sum()
        
    else:
        group_def = group_def.append(playerData.groupby(by=["nickName","CoverageType_descr"]).sum())
    count += 1
    
group_def.reset_index(inplace=True)
receiving_CoverageType_def = group_def[['nickName','CoverageType_descr','FP_yards', 'FP_TD','FP_receiving','FP_total','plays']]


In [224]:
# SIS COVERAGE TYPE GROUPED BY TEAM
dfRead_CoverageType_Team =  dfRead_CoverageType.groupby(by=["Team", 'CoverageType']).sum()
dfRead_CoverageType_Team.reset_index(inplace=True)

In [225]:
# Merge receiving data with coverage type routes run.

receiving_CoverageType_routesRun = pd.merge(receiving_CoverageType_def, 
                                dfRead_CoverageType_Team[['Team','CoverageType','Routes Run']], 
                                left_on = ['nickName','CoverageType_descr'], 
                          right_on=['Team','CoverageType', ], how = 'left')

receiving_CoverageType_routesRun = receiving_CoverageType_routesRun.dropna(subset = ['Routes Run'])

receiving_CoverageType_routesRun['FP/routes'] = receiving_CoverageType_routesRun['FP_total']/receiving_CoverageType_routesRun['Routes Run']

receiving_CoverageType_routesRun = receiving_CoverageType_routesRun[['nickName','CoverageType_descr','FP_yards','FP_TD', 'FP_receiving' , 'FP_total', 'Routes Run', 'FP/routes','plays']]



In [226]:
# Calculte frequency

totalPlayCountTable = receiving_CoverageType_routesRun.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_CoverageType_routesRun = pd.merge(receiving_CoverageType_routesRun, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
receiving_CoverageType_routesRun['freq'] = receiving_CoverageType_routesRun['plays'] / receiving_CoverageType_routesRun['totalPlays']


In [1119]:
receiving_CoverageType_routesRun.to_csv('Split/Receiving_CoverageType_def_2020_0817.csv', index = False)


<h5> FP/route group by defense coverage - POOLED <h5>

In [1121]:

count = 0
for i in np.unique(receiving_pbp_coverage['playerName']):
    playerData = receiving_pbp_coverage[receiving_pbp_coverage['playerName'] == i]
    if count == 0:
        group = playerData.groupby(by=["team", "playerName","CoverageType_pooled"]).sum()
        
    else:
        group = group.append(playerData.groupby(by=["team", "playerName","CoverageType_pooled"]).sum())
    count += 1
    
group.reset_index(inplace=True)
receiving_CoverageType_pooled = group[["team", 'playerName','CoverageType_pooled','FP_yards', 'FP_TD','FP_receiving','FP_total', 'plays']]

    

In [1123]:
# SIS dashboard coverage type routes run POOLED

CoverageType_pooled = []

for i in range(len(dfRead_CoverageType)):
    if dfRead_CoverageType['CoverageType'][i] in ['Cover 0', 'Cover 1', 'Cover 2', 'Man Cover 2']:
        CoverageType_pooled.append("Man")
    elif dfRead_CoverageType['CoverageType'][i] in ['Cover 3','Cover 4', 'Cover 6']:
        CoverageType_pooled.append("Zone")
    else:
        CoverageType_pooled.append("Other")
    
dfRead_CoverageType['CoverageType_pooled'] = CoverageType_pooled

# Pooling
dfRead_CoverageType = dfRead_CoverageType.groupby(by=["Season", "Player", "Team","CoverageType_pooled"]).sum()
dfRead_CoverageType.reset_index(inplace=True)


In [1124]:
# Merge receiving data with coverage type routes run - POOLED

receiving_CoverageType_routesRun_pooled = pd.merge(receiving_CoverageType_pooled, 
                                dfRead_CoverageType[['Player','CoverageType_pooled','Routes Run']], 
                                left_on = ['playerName','CoverageType_pooled'], 
                          right_on=['Player','CoverageType_pooled', ], how = 'left')

receiving_CoverageType_routesRun_pooled = receiving_CoverageType_routesRun_pooled.dropna(subset = ['Routes Run'])

receiving_CoverageType_routesRun_pooled['FP/routes'] = receiving_CoverageType_routesRun_pooled['FP_total']/receiving_CoverageType_routesRun_pooled['Routes Run']

receiving_CoverageType_routesRun_pooled = receiving_CoverageType_routesRun_pooled[["team", 'playerName','CoverageType_pooled','FP_yards','FP_TD', 'FP_receiving' , 'FP_total', 'Routes Run', 'FP/routes', 'plays']]


In [1125]:
# Calculte frequency

totalPlayCountTable = receiving_CoverageType_routesRun_pooled.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_CoverageType_routesRun_pooled = pd.merge(receiving_CoverageType_routesRun_pooled, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
receiving_CoverageType_routesRun_pooled['freq'] = receiving_CoverageType_routesRun_pooled['plays'] / receiving_CoverageType_routesRun_pooled['totalPlays']



In [1126]:

receiving_CoverageType_routesRun_pooled.to_csv('Split/Receiving_CoverageType_pooled_2020_0817.csv', index = False)

<h5> FP/route group by defense coverage - POOLED - defensive view <h5>

In [1127]:
# Merge PBP with opponent teams 

receiving_pbp_coverage_OPP = pd.merge(receiving_pbp_coverage, preSnapDefenseTeam[['season', 'week', 'gameId','oppTeamId', 'nickName']], 
         left_on = ['season', 'week', 'gameId'], 
                          right_on=['season', 'week', 'gameId'], how = 'left')
receiving_pbp_coverage_OPP = receiving_pbp_coverage_OPP[receiving_pbp_coverage_OPP['teamId'] != receiving_pbp_coverage_OPP['oppTeamId']].reset_index(drop = True)


In [1128]:
# FP/route by defense coverage - defensive teams

count = 0
for i in np.unique(receiving_pbp_coverage_OPP['nickName']):
    playerData = receiving_pbp_coverage_OPP[receiving_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        group_def = playerData.groupby(by=["nickName","CoverageType_pooled"]).sum()
        
    else:
        group_def = group_def.append(playerData.groupby(by=["nickName","CoverageType_pooled"]).sum())
    count += 1
    
group_def.reset_index(inplace=True)
receiving_CoverageType_def_pooled = group_def[['nickName','CoverageType_pooled','FP_yards', 'FP_TD','FP_receiving','FP_total', 'plays']]


In [1129]:
# SIS DOWNLOAD COVERAGE TYPE GROUPING BY TEAM
dfRead_CoverageType_Team_pooled =  dfRead_CoverageType.groupby(by=["Team", 'CoverageType_pooled']).sum()
dfRead_CoverageType_Team_pooled.reset_index(inplace=True)

In [1130]:
# Merge receiving data with coverage type routes run.


receiving_CoverageType_routesRun_def_pooled = pd.merge(receiving_CoverageType_def_pooled, 
                                dfRead_CoverageType_Team_pooled[['Team','CoverageType_pooled','Routes Run']], 
                                left_on = ['nickName','CoverageType_pooled'], 
                          right_on=['Team','CoverageType_pooled', ], how = 'left')

receiving_CoverageType_routesRun_def_pooled = receiving_CoverageType_routesRun_def_pooled.dropna(subset = ['Routes Run'])

receiving_CoverageType_routesRun_def_pooled['FP/routes'] = receiving_CoverageType_routesRun_def_pooled['FP_total']/receiving_CoverageType_routesRun_def_pooled['Routes Run']

receiving_CoverageType_routesRun_def_pooled = receiving_CoverageType_routesRun_def_pooled[['nickName','CoverageType_pooled','FP_yards','FP_TD', 'FP_receiving' , 'FP_total', 'Routes Run', 'FP/routes', 'plays']]


In [1131]:
# Calculte frequency

totalPlayCountTable = receiving_CoverageType_routesRun_def_pooled.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_CoverageType_routesRun_def_pooled = pd.merge(receiving_CoverageType_routesRun_def_pooled, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
receiving_CoverageType_routesRun_def_pooled['freq'] = receiving_CoverageType_routesRun_def_pooled['plays'] / receiving_CoverageType_routesRun_def_pooled['totalPlays']


In [1132]:

receiving_CoverageType_routesRun_def_pooled.to_csv('Split/Receiving_CoverageType_def_pooled_2020_0817.csv', index = False)


<h3> By Drop Type <h3>

In [228]:
# By Drop Type
count = 0
for i in np.unique(receiving_pbp_coverage['playerName']):
    playerData = receiving_pbp_coverage[receiving_pbp_coverage['playerName'] == i]
    if count == 0:
        group = playerData.groupby(by=["team", "playerName","DropType_descr"]).sum()
        
    else:
        group = group.append(playerData.groupby(by=["team", "playerName","DropType_descr"]).sum())
    count += 1
    
group.reset_index(inplace=True)


#group['FP/routes'] = group['FP_total']/group['routesRun']


#receiving_DropType = group[['playerId','DropType_descr','routesRun','FP_yards', 'FP_TD','FP_receiving','FP_total','FP/routes']]


receiving_DropType = group[["team", 'playerName','DropType_descr','FP_yards', 'FP_TD','FP_receiving','FP_total', 'plays']]



In [231]:
# Load all routes by by drop types (downloads from SIS dashboard)
#directory = 'SIS Dashboard/2020Archive/DropType/'
directory = 'SIS Dashboard/DropType/'
count = 0
for filename in os.listdir(directory):
    if 'csv' not in filename:
        continue
    f = os.path.join(directory, filename)
    fname = filename[:filename.index('.csv')]
    if count == 0:
        dfRead_DropType = pd.read_csv(f) 
        dfRead_DropType['DropType'] = fname
    else:
        dfRead_DropType_tmp = pd.read_csv(f)
        dfRead_DropType_tmp['DropType'] = fname
        dfRead_DropType = dfRead_DropType.append(dfRead_DropType_tmp, ignore_index=True)
    count += 1
    
DropType = []
for i in range(len(dfRead_DropType)):
    if  dfRead_DropType['DropType'][i] == "0_1 Step":
        DropType.append("0/1 Step")
    else:
        DropType.append(dfRead_DropType['DropType'][i])        
dfRead_DropType['DropType'] = DropType     

In [232]:
# Merge receiving data with drop type routes run.

receiving_DropType_routesRun = pd.merge(receiving_DropType, 
                                dfRead_DropType[['Player','DropType','Routes Run']], 
                                left_on = ['playerName','DropType_descr'], 
                          right_on=['Player','DropType', ], how = 'left')

receiving_DropType_routesRun = receiving_DropType_routesRun.dropna(subset = ['Routes Run'])

receiving_DropType_routesRun['FP/routes'] = receiving_DropType_routesRun['FP_total']/receiving_DropType_routesRun['Routes Run']

receiving_DropType_routesRun = receiving_DropType_routesRun[["team", 'playerName','Routes Run','DropType_descr','FP_yards','FP_TD', 'FP_receiving' , 'FP_total', 'Routes Run', 'FP/routes', 'plays']]


In [233]:
# Calculte frequency

totalPlayCountTable = receiving_DropType_routesRun.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_DropType_routesRun = pd.merge(receiving_DropType_routesRun, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
receiving_DropType_routesRun['freq'] = receiving_DropType_routesRun['plays'] / receiving_DropType_routesRun['totalPlays']


In [1138]:
receiving_DropType_routesRun.to_csv('Split/Receiving_DropType_2020_0817.csv', index = False)

<h5> Receiving By Drop Type - defense<h5>

In [1139]:
# FP/route by defense coverage - defensive teams

count = 0
for i in np.unique(receiving_pbp_coverage_OPP['nickName']):
    playerData = receiving_pbp_coverage_OPP[receiving_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        group_def = playerData.groupby(by=["nickName","DropType_descr"]).sum()

    else:
        group_def = group_def.append(playerData.groupby(by=["nickName","DropType_descr"]).sum())
    count += 1
    
group_def.reset_index(inplace=True)
receiving_DropType_def = group_def[['nickName','DropType_descr','FP_yards', 'FP_TD','FP_receiving','FP_total', 'plays']]


In [1140]:
# SIS DOWNLOAD DROP TYPE GROUPING BY TEAM
dfRead_DropType_Team =  dfRead_DropType.groupby(by=["Team", 'DropType']).sum()
dfRead_DropType_Team.reset_index(inplace=True)

In [1141]:
# Merge receiving data with drop type routes run - Defense

receiving_DropType_routesRun = pd.merge(receiving_DropType_def, 
                                dfRead_DropType_Team[['Team','DropType','Routes Run']], 
                                left_on = ['nickName','DropType_descr'], 
                          right_on=['Team','DropType', ], how = 'left')

receiving_DropType_routesRun = receiving_DropType_routesRun.dropna(subset = ['Routes Run'])

receiving_DropType_routesRun['FP/routes'] = receiving_DropType_routesRun['FP_total']/receiving_DropType_routesRun['Routes Run']

receiving_DropType_routesRun = receiving_DropType_routesRun[['nickName','Routes Run','DropType_descr','FP_yards','FP_TD', 'FP_receiving' , 'FP_total', 'Routes Run', 'FP/routes', 'plays']]


In [1142]:
# Calculte frequency

totalPlayCountTable = receiving_DropType_routesRun.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_DropType_routesRun = pd.merge(receiving_DropType_routesRun, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
receiving_DropType_routesRun['freq'] = receiving_DropType_routesRun['plays'] / receiving_DropType_routesRun['totalPlays']


In [1143]:

receiving_DropType_routesRun.to_csv('Split/Receiving_DropType_def_2020_0817.csv', index = False)


<h5> RECEIVING FP/Routes by route type  <h5>

In [236]:
#By Route Type
count = 0
for i in np.unique(receiving_pbp_coverage['playerName']):
    playerData = receiving_pbp_coverage[receiving_pbp_coverage['playerName'] == i]
    if count == 0:
        group = playerData.groupby(by=["team","playerName","TargetType_descr"]).sum()
        
    else:
        group = group.append(playerData.groupby(by=["team","playerName","TargetType_descr"]).sum())
    count += 1
    
group.reset_index(inplace=True)


receiving_TargetType = group[["team",'playerName','TargetType_descr','FP_yards','FP_TD','FP_receiving','FP_total','plays']]



In [237]:
# Load all routes by by route types (downloads from SIS dashboard)
#directory = 'SIS Dashboard/2020Archive/RouteType/'
directory = 'SIS Dashboard/RouteType/'
count = 0
for filename in os.listdir(directory):
    if 'csv' not in filename:
        continue
    f = os.path.join(directory, filename)
    fname = filename[:filename.index('.csv')]
    if count == 0:
        dfRead_RouteType = pd.read_csv(f) 
        dfRead_RouteType['RouteType'] = fname
    else:
        dfRead_RouteType_tmp = pd.read_csv(f)
        dfRead_RouteType_tmp['RouteType'] = fname
        dfRead_RouteType = dfRead_RouteType.append(dfRead_RouteType_tmp, ignore_index=True)

    count += 1


In [238]:
# Merge receiving data with route types routes run.

receiving_pbp_coverage_routetype = pd.merge(receiving_TargetType, 
                                dfRead_RouteType[['Player','RouteType','Routes Run']], 
                                left_on = ['playerName','TargetType_descr'], 
                          right_on=['Player','RouteType', ], how = 'left')

receiving_pbp_coverage_routetype = receiving_pbp_coverage_routetype.dropna(subset = ['Routes Run'])

receiving_pbp_coverage_routetype['FP/routes'] = receiving_pbp_coverage_routetype['FP_total']/receiving_pbp_coverage_routetype['Routes Run']



In [247]:
# Post process map to target type NUMBERING

receiving_RouteTree = receiving_pbp_coverage_routetype.reset_index(drop = True).copy()
routeTree = pd.read_csv("RouteTree.csv", index_col = False)

targetTypeMap = []
for i in range(len(receiving_RouteTree)):
    try:
        ind = receiving_RouteTree['TargetType_descr'][i].index("-")
        targetTypeMap.append(receiving_RouteTree['TargetType_descr'][i][:ind - 1])
    except:
        targetTypeMap.append(receiving_RouteTree['TargetType_descr'][i])
receiving_RouteTree['targetTypeMap'] = targetTypeMap

receiving_RouteTree = pd.merge(receiving_RouteTree, routeTree[['All', 'SIS']], left_on = ['targetTypeMap'],
                          right_on=['All'], how = 'left')

receiving_RouteTree = receiving_RouteTree.rename(columns={"SIS" : "route"})
receiving_RouteTree = receiving_RouteTree.drop(columns = ['All'])

In [262]:
receiving_RouteTree = receiving_RouteTree[["team",'playerName','route','FP_yards','FP_TD','FP_receiving','FP/routes', 'plays']]


In [249]:
# Calculte frequency

totalPlayCountTable = receiving_RouteTree.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_RouteTree = pd.merge(receiving_RouteTree, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
receiving_RouteTree['freq'] = receiving_RouteTree['plays'] / receiving_RouteTree['totalPlays']



In [1150]:

receiving_RouteTree.to_csv('Split/Receiving_RouteType_2020_0817.csv', index = False)

<h5> Receiving By Route Type - defense <h5>

In [1151]:
# FP/route by route type - defensive teams
count = 0
for i in np.unique(receiving_pbp_coverage_OPP['nickName']):
    playerData = receiving_pbp_coverage_OPP[receiving_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        group = playerData.groupby(by=["nickName","TargetType_descr"]).sum()
        
    else:
        group = group.append(playerData.groupby(by=["nickName","TargetType_descr"]).sum())
    count += 1

group.reset_index(inplace=True)


receiving_TargetType_def = group[['nickName','TargetType_descr','FP_yards','FP_TD','FP_receiving','FP_total', 'plays']]

In [1152]:
# SIS DOWNLOAD ROUTE TYPE GROUPING BY TEAM
dfRead_RouteType_Team =  dfRead_RouteType.groupby(by=["Team", 'RouteType']).sum()
dfRead_RouteType_Team.reset_index(inplace=True)

In [1153]:
# Merge receiving data with route types routes run - defense

receiving_pbp_coverage_routetype = pd.merge(receiving_TargetType_def, 
                                dfRead_RouteType_Team[['Team','RouteType','Routes Run']], 
                                left_on = ['nickName','TargetType_descr'], 
                          right_on=['Team','RouteType', ], how = 'left')

receiving_pbp_coverage_routetype = receiving_pbp_coverage_routetype.dropna(subset = ['Routes Run'])

receiving_pbp_coverage_routetype['FP/routes'] = receiving_pbp_coverage_routetype['FP_total']/receiving_pbp_coverage_routetype['Routes Run']


In [568]:
#receiving_pbp_coverage_routetype.to_csv('Split/Receiving_RouteType_def_0806.csv')

In [1154]:
# Post processing map to route NUMBERING


receiving_RouteTree = receiving_pbp_coverage_routetype.reset_index(drop = True).copy()
routeTree = pd.read_csv("RouteTree.csv", index_col = False)

targetTypeMap = []
for i in range(len(receiving_RouteTree)):
    try:
        ind = receiving_RouteTree['TargetType_descr'][i].index("-")
        targetTypeMap.append(receiving_RouteTree['TargetType_descr'][i][:ind - 1])
    except:
        targetTypeMap.append(receiving_RouteTree['TargetType_descr'][i])
receiving_RouteTree['targetTypeMap'] = targetTypeMap

receiving_RouteTree = pd.merge(receiving_RouteTree, routeTree[['All', 'SIS']], left_on = ['targetTypeMap'],
                          right_on=['All'], how = 'left')

receiving_RouteTree = receiving_RouteTree.rename(columns={"SIS" : "route"})
receiving_RouteTree = receiving_RouteTree.drop(columns = ['All'])

receiving_RouteTree = receiving_RouteTree[["nickName",'route','FP_yards','FP_TD','FP_receiving','FP/routes', 'plays']]


In [1155]:
# Calculte frequency

totalPlayCountTable = receiving_RouteTree.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
receiving_RouteTree = pd.merge(receiving_RouteTree, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
receiving_RouteTree['freq'] = receiving_RouteTree['plays'] / receiving_RouteTree['totalPlays']


In [1156]:

receiving_RouteTree.to_csv('Split/Receiving_RouteType_def_2020_0817.csv', index = False)

In [ ]:
#FP/route run for receivers and TE for the following:

#for WRs/TEs
#man and zone (coverage type to be more nuanced as well)
#Drop Type
#Route Type
#inside vs outside, IE what were his fp/route run in the slot vs outside slot

# Rushing

In [1157]:
# Merge receiving with coverage types
rushing_pbp_coverage = pd.merge(rushing_pbp_map, 
                                presnapDefense_pbp_map[['season', 'week', 'gameId', 'eventId', 'coverageSchemeType', 'CoverageType_descr']], 
                                left_on = ['season', 'week', 'gameId', 'eventId'], 
                          right_on=['season', 'week', 'gameId', 'eventId'], how = 'left')


In [1158]:
#2021 only
#rushing_pbp_coverage= rushing_pbp_coverage[rushing_pbp_coverage['season'] == 2021].reset_index(drop = True)

rushing_pbp_coverage= rushing_pbp_coverage[rushing_pbp_coverage['season'] == 2020].reset_index(drop = True)


In [1159]:
# Add FP yards and touchdown
rushing_pbp_coverage['FP_yards'] = rushing_pbp_coverage['yards'] * 0.1

FP_TD= []
#passing_pbp_coverage['FP_TD'] = 0
for i in range(len(rushing_pbp_coverage)):
    if rushing_pbp_coverage['scoreType'][i] == 6:
        FP_TD.append(6)
    else:
        FP_TD.append(0)
rushing_pbp_coverage['FP_TD'] = FP_TD

In [1160]:
# FP rushing
FP_fumbleLost = []

for i in range(len(rushing_pbp_coverage)):
    if rushing_pbp_coverage['fumbleLost'][i] == False:
        FP_fumbleLost.append(0)
    else: 
        FP_fumbleLost.append(-1)
rushing_pbp_coverage['FP_fumbleLost'] = FP_fumbleLost

In [1161]:
rushing_pbp_coverage['FP_total'] = rushing_pbp_coverage['FP_yards'] +  rushing_pbp_coverage['FP_TD'] + rushing_pbp_coverage['FP_fumbleLost']



In [1162]:
# define a touch

touch = []
for i in range(len(rushing_pbp_coverage)):
    if rushing_pbp_coverage['ActionType_descr'][i] == "Carry":
        touch.append(1)
    else:
        touch.append(0)

rushing_pbp_coverage['touch'] = touch

In [1163]:
rushing_pbp_coverage['plays'] = 1

In [1164]:
blockingSchemeType_pooled = []

for i in range(len(rushing_pbp_coverage)):
    if ('pull' in str(rushing_pbp_coverage['blockingSchemeType_descr'][i])) | (rushing_pbp_coverage['blockingSchemeType_descr'][i] == 'Duo'):
        blockingSchemeType_pooled.append("Gap")
    elif (rushing_pbp_coverage['blockingSchemeType_descr'][i] == 'Zone') | (rushing_pbp_coverage['blockingSchemeType_descr'][i] == 'Split Zone'):
        blockingSchemeType_pooled.append("Zone")
    else:
        blockingSchemeType_pooled.append("Other")

rushing_pbp_coverage['blockingSchemeType_pooled'] = blockingSchemeType_pooled        

#Gap: Anything with "pull" in the name (including "false pull"), as well as Duo
#Zone:Zone, Split Zone
#Other: Everything else


In [1166]:
# Map rushing to deep chart - LONG RUNTIME
deepChart = []
team = []
for i in range(len(rushing_pbp_coverage)):
    inDeepChart = 0
    team_tmp = 'team'
    for j in deepChartOffense.columns:
        for k in range(len(deepChartOffense)):
            if rushing_pbp_coverage['playerName'][i] == deepChartOffense[j][k]:
                inDeepChart += 1
                team_tmp = j
    if inDeepChart != 0:
        deepChart.append(1)
        team.append(team_tmp)
    else:
        deepChart.append(0)
        team.append("")
rushing_pbp_coverage['deepChart'] = deepChart
rushing_pbp_coverage['team'] = team
rushing_pbp_coverage = rushing_pbp_coverage[rushing_pbp_coverage['deepChart'] == 1].reset_index(drop = True)


In [1170]:
# Group by block scheme

count = 0
for i in np.unique(rushing_pbp_coverage['playerName']):
    playerData = rushing_pbp_coverage[rushing_pbp_coverage['playerName'] == i]
    if count == 0:
        group = playerData.groupby(by=["team", "playerName","blockingSchemeType_descr"]).sum()

    else:
        group = group.append(playerData.groupby(by=["team", "playerName","blockingSchemeType_descr"]).sum())
    count += 1
    
group.reset_index(inplace=True)

group['FP/Touch'] = group['FP_total']/group['touch']

group_rushing_block = group[["team", 'playerName','blockingSchemeType_descr','touch', 'FP_yards', 'FP_TD','FP_fumbleLost','FP_total','FP/Touch','plays']]



In [1171]:
#MINIMUM CONDITION
group_rushing_block = group_rushing_block[group_rushing_block['touch'] > 5].reset_index(drop = True)

In [1172]:
# Calculte frequency

totalPlayCountTable = group_rushing_block.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
group_rushing_block = pd.merge(group_rushing_block, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
group_rushing_block['freq'] = group_rushing_block['plays'] / group_rushing_block['totalPlays']


<h5> Group by block schemes - defensive teams <h5>

In [1174]:
# Merge with opponent teams

rushing_pbp_coverage_OPP = pd.merge(rushing_pbp_coverage, preSnapDefenseTeam[['season', 'week', 'gameId','oppTeamId', 'nickName']], 
         left_on = ['season', 'week', 'gameId'], 
                          right_on=['season', 'week', 'gameId'], how = 'left')
rushing_pbp_coverage_OPP = rushing_pbp_coverage_OPP[rushing_pbp_coverage_OPP['teamId'] != rushing_pbp_coverage_OPP['oppTeamId']].reset_index(drop = True)



In [526]:
#rushing_pbp_coverage_OPP.to_csv("NN_Data/Defense/rushing_pbp_OPP_081622.csv")

In [1175]:
# Group by all block scheme - defense

count = 0
for i in np.unique(rushing_pbp_coverage_OPP['nickName']):
    playerData = rushing_pbp_coverage_OPP[rushing_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        group_def = playerData.groupby(by=["nickName","blockingSchemeType_descr"]).sum()

    else:
        group_def = group_def.append(playerData.groupby(by=["nickName","blockingSchemeType_descr"]).sum())
    count += 1
    
group_def.reset_index(inplace=True)

group_def['FP/Touch'] = group_def['FP_total']/group_def['touch']

group_rushing_block_def = group_def[['nickName','blockingSchemeType_descr','touch', 'FP_yards', 'FP_TD','FP_fumbleLost','FP_total','FP/Touch', 'plays']]



In [1176]:
group_rushing_block_def = group_rushing_block_def[group_rushing_block_def['touch'] > 10].reset_index(drop = True)

In [1177]:
# Calculte frequency

totalPlayCountTable = group_rushing_block_def.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
group_rushing_block_def = pd.merge(group_rushing_block_def, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
group_rushing_block_def['freq'] = group_rushing_block_def['plays'] / group_rushing_block_def['totalPlays']


In [1182]:

group_rushing_block.to_csv('Split/Rushing_BlockingScheme_2020_0817.csv', index = False)
group_rushing_block_def.to_csv('Split/Rushing_Blocking_def_2020_0817.csv',  index = False)

<h5> Group by block schemes - pooled <h5>

In [1184]:
# Group by all block scheme

count = 0
for i in np.unique(rushing_pbp_coverage['playerName']):
    playerData = rushing_pbp_coverage[rushing_pbp_coverage['playerName'] == i]
    if count == 0:
        group = playerData.groupby(by=["team", "playerName","blockingSchemeType_pooled"]).sum()

    else:
        group = group.append(playerData.groupby(by=["team", "playerName","blockingSchemeType_pooled"]).sum())
    count += 1

group.reset_index(inplace=True)

group['FP/Touch'] = group['FP_total']/group['touch']

group_rushing_block_pooled = group[["team", 'playerName','blockingSchemeType_pooled','touch', 'FP_yards', 'FP_TD','FP_fumbleLost','FP_total','FP/Touch', 'plays']]


In [1185]:
group_rushing_block_pooled = group_rushing_block_pooled[group_rushing_block_pooled['touch'] > 5].reset_index(drop = True)



In [1186]:
# Calculte frequency

totalPlayCountTable = group_rushing_block_pooled.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
group_rushing_block_pooled = pd.merge(group_rushing_block_pooled, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
group_rushing_block_pooled['freq'] = group_rushing_block_pooled['plays'] / group_rushing_block_pooled['totalPlays']

In [1187]:
group_rushing_block_pooled.to_csv('Split/Rushing_BlockingScheme_pooled_2020_0817.csv', index = 'False')

<h5> Group by block schemes defensive team - pooled <h5>

In [1190]:
# Merge with opponent teams

rushing_pbp_coverage_OPP_pooled = pd.merge(rushing_pbp_coverage, preSnapDefenseTeam[['season', 'week', 'gameId','oppTeamId', 'nickName']], 
         left_on = ['season', 'week', 'gameId'], 
                          right_on=['season', 'week', 'gameId'], how = 'left')
rushing_pbp_coverage_OPP_pooled = rushing_pbp_coverage_OPP_pooled[rushing_pbp_coverage_OPP_pooled['teamId'] != rushing_pbp_coverage_OPP_pooled['oppTeamId']].reset_index(drop = True)




In [1191]:
# Group by all block scheme - defense

count = 0
for i in np.unique(rushing_pbp_coverage_OPP_pooled['nickName']):
    playerData = rushing_pbp_coverage_OPP_pooled[rushing_pbp_coverage_OPP_pooled['nickName'] == i]
    if count == 0:
        group_def = playerData.groupby(by=["nickName","blockingSchemeType_pooled"]).sum()

    else:
        group_def = group_def.append(playerData.groupby(by=["nickName","blockingSchemeType_pooled"]).sum())
    count += 1
    
group_def.reset_index(inplace=True)

group_def['FP/Touch'] = group_def['FP_total']/group_def['touch']

group_rushing_block_def_pooled = group_def[['nickName','blockingSchemeType_pooled','touch', 'FP_yards', 'FP_TD','FP_fumbleLost','FP_total','FP/Touch', 'plays']]



In [1192]:
# Calculte frequency

totalPlayCountTable = group_rushing_block_def_pooled.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
group_rushing_block_def_pooled = pd.merge(group_rushing_block_def_pooled, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
group_rushing_block_def_pooled['freq'] = group_rushing_block_def_pooled['plays'] / group_rushing_block_def_pooled['totalPlays']


In [1193]:
group_rushing_block_def_pooled.to_csv('Split/Rushing_BlockingScheme_def_pooled_2020_0817.csv', index = 'False')

In [1194]:
# Group by direction

count = 0
for i in np.unique(rushing_pbp_coverage['playerName']):
    playerData = rushing_pbp_coverage[rushing_pbp_coverage['playerName'] == i]
    if count == 0:
        group = playerData.groupby(by=["team", "playerName","RunDirection_descr"]).sum()
        
    else:
        group = group.append(playerData.groupby(by=["team", "playerName","RunDirection_descr"]).sum())
    count += 1
    
group.reset_index(inplace=True)

group['FP/Touch'] = group['FP_total']/group['touch']

group_rushing_direction = group[["team", 'playerName','RunDirection_descr','touch', 'FP_yards', 'FP_TD','FP_fumbleLost','FP_total','FP/Touch', 'plays']]


In [1196]:
#Minimum condition
group_rushing_direction = group_rushing_direction[group_rushing_direction['touch'] > 5].reset_index(drop = True)

In [1197]:
# Calculte frequency

totalPlayCountTable = group_rushing_direction.groupby(by=["playerName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
group_rushing_direction = pd.merge(group_rushing_direction, totalPlayCountTable[['playerName', 'totalPlays']], 
         left_on = ['playerName'], 
         right_on=['playerName'], how = 'left')
group_rushing_direction['freq'] = group_rushing_direction['plays'] / group_rushing_direction['totalPlays']


In [1198]:
group_rushing_direction.to_csv('Split/Rushing_direction_2020_0817.csv', index = False)

<h5> Group by direction - defense <h5>

In [1199]:
# Group by direction - defense

count = 0
for i in np.unique(rushing_pbp_coverage_OPP['nickName']):
    playerData = rushing_pbp_coverage_OPP[rushing_pbp_coverage_OPP['nickName'] == i]
    if count == 0:
        group_def = playerData.groupby(by=["nickName","RunDirection_descr"]).sum()
        
    else:
        group_def = group_def.append(playerData.groupby(by=["nickName","RunDirection_descr"]).sum())
    count += 1
    
group_def.reset_index(inplace=True)

group_def['FP/Touch'] = group_def['FP_total']/group_def['touch']

group_rushing_direction_def = group_def[['nickName', 'touch','RunDirection_descr','FP_yards', 'FP_TD','FP_fumbleLost','FP_total','FP/Touch', 'plays']]


In [1200]:
group_rushing_direction_def = group_rushing_direction_def[group_rushing_direction_def['touch'] > 10].reset_index(drop = True)


In [1201]:
# Calculte frequency

totalPlayCountTable = group_rushing_direction_def.groupby(by=["nickName"]).sum()
totalPlayCountTable.reset_index(inplace=True)
totalPlayCountTable = totalPlayCountTable.rename(columns={"plays" : "totalPlays"})
group_rushing_direction_def = pd.merge(group_rushing_direction_def, totalPlayCountTable[['nickName', 'totalPlays']], 
         left_on = ['nickName'], 
         right_on=['nickName'], how = 'left')
group_rushing_direction_def['freq'] = group_rushing_direction_def['plays'] / group_rushing_direction_def['totalPlays']


In [1202]:
group_rushing_direction_def.to_csv('Split/Rushing_Direction_def_2020_0817.csv')